In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('pbp_raw/0022300142_pbp.csv',index_col=0)

def str_to_tuple(s):
    return tuple(s.strip('() ').replace(' ', '').split(','))

df['h_lineup'] = df.h_lineup.apply(lambda x: str_to_tuple(x))
df['a_lineup'] = df.a_lineup.apply(lambda x: str_to_tuple(x))

eventdict = {1:'FGM',
             2:'FGA',
             3:'FTA',
             4:'TRB',
             5:'TURNOVER',
             6:'FOUL',
             8:'SUB',
             9:'TIMEOUT',
             10:'JUMP_BALL',
             12:'Q_START',
             13:'Q_END',
             18:'REPLAY'             
            }
cols = ['game_id', 'lineup','player_id','PTS','FGM', 'FGA', '3PM', '3PA', 'FTM', 'FTA', 'REB', 'OREB', 'DREB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'MIN', 'POSS']


In [2]:
#allll this ugly stuff just to get a column called 'rebound_type'
mask = (df.eventmsgtype == 4) & (~df.homedescription.isnull() | ~df.visitordescription.isnull())
pattern = r'Off:(\d+) Def:(\d+)'

home_extract = df['homedescription'].str.extract(pattern)
df['offensive_rebounds'] = home_extract[0]
df['defensive_rebounds'] = home_extract[1]

visitor_extract = df['visitordescription'].str.extract(pattern)
df['offensive_rebounds'].fillna(visitor_extract[0], inplace=True)
df['defensive_rebounds'].fillna(visitor_extract[1], inplace=True)

df[['offensive_rebounds', 'defensive_rebounds']] = df[['offensive_rebounds', 'defensive_rebounds']].fillna(0)
df[['offensive_rebounds', 'defensive_rebounds']] = df[['offensive_rebounds', 'defensive_rebounds']].astype(int)
df['def_reb_previous'] = df[mask].groupby('player1_id')['defensive_rebounds'].shift(fill_value=0)
df['off_reb_previous'] = df[mask].groupby('player1_id')['offensive_rebounds'].shift(fill_value=0)
df.sort_values(by=['period','total_elapsed_time','eventnum'],inplace=True)
df['rebound_type'] = ''
df.loc[(df.defensive_rebounds > df.def_reb_previous), 'rebound_type'] = 'DRB'
df.loc[(df.offensive_rebounds > df.off_reb_previous), 'rebound_type'] = 'ORB'
df.drop([ 'offensive_rebounds', 'defensive_rebounds', 'def_reb_previous', 'off_reb_previous'],inplace=True,axis=1)
df.sort_values(by=['period','total_elapsed_time','eventnum'],inplace=True)

In [3]:
fga_filter = (df.eventmsgtype==1) | (df.eventmsgtype==2)
two_pt_filter = (df.eventmsgtype==1) & ~((df.homedescription.str.contains('3PT',na=False)) | (df.visitordescription.str.contains('3PT',na=False)))
three_pt_filter = (df.eventmsgtype==1) & ((df.homedescription.str.contains('3PT',na=False)) | (df.visitordescription.str.contains('3PT',na=False)))
three_a_filter = ((df.homedescription.str.contains('3PT',na=False)) | (df.visitordescription.str.contains('3PT',na=False)))
ft_filter = (df.eventmsgtype==3) & ((df.homedescription.str.contains('PTS',na=False)) | (df.visitordescription.str.contains('PTS',na=False)))
fta_filter = df.eventmsgtype==3 
assist_filter = (df.eventmsgtype==1) & ~df.player2_name.isnull()

orb_filter = df.rebound_type == 'ORB'
drb_filter = df.rebound_type == 'DRB'
trb_filter = (df.rebound_type == 'DRB') | (df.rebound_type == 'ORB')
stl_filter = df.homedescription.str.contains('STEAL',na=False) | df.visitordescription.str.contains('STEAL',na=False)
blk_filter = df.homedescription.str.contains('BLOCK',na=False) | df.visitordescription.str.contains('BLOCK',na=False)
to_filter = df.eventmsgtype==5
foul_filter = df.eventmsgtype==6
time_filter = (df.play_elapsed_time != 0)

In [4]:
made_last_ft_pattern = r'(1 of 1|2 of 2|3 of 3).*PTS'
visitor_poss_filter = (
((df.rebound_type == 'DRB') & ~(df.homedescription.isna())) | # home team drb means visitor poss ends
((df.eventmsgtype == 4) & ~(df.eventmsgactiontype==1) & (~df.homedescription.isna())) |
(df.homedescription.str.contains('STEAL',na=False)) | # home team steal means visitor poss ends
((df.eventmsgtype==1) & ~(df.visitordescription.isna())) | # visitor team makes shot means visitor poss ends
((df.eventmsgtype==3) & df.visitordescription.str.contains(made_last_ft_pattern)) | #visitor team MAKES last ft means poss ends
((df.eventmsgtype == 5) & (df.homedescription.isna())) #unforced visitor turnover
) 

home_poss_filter = (
((df.rebound_type == 'DRB') & ~(df.visitordescription.isna())) | # home team drb means visitor poss ends
((df.eventmsgtype == 4) & ~(df.eventmsgactiontype==1) & (~df.visitordescription.isna())) |
(df.visitordescription.str.contains('STEAL',na=False)) |
((df.eventmsgtype==1) & ~(df.homedescription.isna())) |
((df.eventmsgtype==3) & df.homedescription.str.contains(made_last_ft_pattern)) |
((df.eventmsgtype == 5) & (df.visitordescription.isna()))
) 


C:\Users\Shared.Westover\AppData\Local\Temp\ipykernel_15200\2033246774.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ((df.eventmsgtype==3) & df.visitordescription.str.contains(made_last_ft_pattern)) | #visitor team MAKES last ft means poss ends
C:\Users\Shared.Westover\AppData\Local\Temp\ipykernel_15200\2033246774.py:16: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ((df.eventmsgtype==3) & df.homedescription.str.contains(made_last_ft_pattern)) |


In [63]:
df['home_poss'],df['away_poss'] = 0,0
df.loc[visitor_poss_filter,'away_poss'] = 1
df.loc[home_poss_filter,'home_poss'] = 1
# Determine the most recent possession
def determine_possession(row):
    if row['home_poss'] == 1:
        return 'home'
    elif row['away_poss'] == 1:
        return 'away'


df['home_poss'],df['away_poss'] = 0,0
df.loc[visitor_poss_filter,'away_poss'] = 1
df.loc[home_poss_filter,'home_poss'] = 1
# Apply the function to each row to create a new 'new_col' column
df['has_ball'] = df.apply(determine_possession, axis=1)

# Fill NaN values in the 'new_col' column based on the previous row
df['has_ball'] = df['has_ball'].fillna(method='bfill')


home_mask = df['has_ball'] == 'home'
home_counter = df.loc[home_mask & (df['has_ball'].shift(-1) == 'away'), 'home_poss'].cumsum()
new_index = range(df.index.min(), df.index.max() + 1)
home_counter = home_counter.reindex(new_index, fill_value=np.NaN).fillna(method='bfill')

# Use the correct DataFrame column to assign values
df.loc[home_mask, 'home_poss'] = home_counter


away_mask = df['has_ball'] == 'away'
away_counter = df.loc[away_mask & (df['has_ball'].shift(-1) == 'home'), 'away_poss'].cumsum()
new_index = range(df.index.min(), df.index.max() + 1)
away_counter = away_counter.reindex(new_index, fill_value=np.NaN).fillna(method='bfill')

# Use the correct DataFrame column to assign values
df.loc[away_mask, 'away_poss'] = away_counter

df.head(1)

,game_id,eventnum,eventmsgtype,eventmsgactiontype,period,pctimestring,homedescription,neutraldescription,visitordescription,score,...,play_elapsed_time,total_elapsed_time,max_time,time_remaining,h_lineup,a_lineup,rebound_type,home_poss,away_poss,has_ball
0,22300142,2,12,0,1,12:00,NaN,Start of 1st Period (6:12 PM EST),NaN,NaN,...,0.0,0.0,2880.0,2880.0,"(1628378, 1628386, 1629622, 1629636, 1630596)","(201939, 202691, 203110, 203952, 1626172)",,0,1.0,away


In [8]:
df.sort_values(by=['period','total_elapsed_time','eventnum'],inplace=True)
df.head(1)

,game_id,eventnum,eventmsgtype,eventmsgactiontype,period,pctimestring,homedescription,neutraldescription,visitordescription,score,...,play_elapsed_time,total_elapsed_time,max_time,time_remaining,h_lineup,a_lineup,rebound_type,home_poss,away_poss,has_ball
0,22300142,2,12,0,1,12:00,NaN,Start of 1st Period (6:12 PM EST),NaN,NaN,...,0.0,0.0,2880.0,2880.0,"(1628378, 1628386, 1629622, 1629636, 1630596)","(201939, 202691, 203110, 203952, 1626172)",,0,1.0,away


In [9]:
df.to_csv('intermediate.csv')

In [37]:
def get_stat_df(pbp_df,filter,stat_type,stat_value,player_num='1'):
    filtered_df = pbp_df.loc[filter][['game_id','eventnum',f'player{player_num}_id',f'player{player_num}_name','h_lineup','a_lineup','home_poss','away_poss']]
    filtered_df.rename(columns={f'player{player_num}_id': 'player_id',
                        f'player{player_num}_name': 'player_name'},inplace=True )
    filtered_df[stat_type] = stat_value
    return filtered_df

In [189]:
#results is a list of every stat credit for every play
results = get_stat_df(df,two_pt_filter,'PTS',2)
results =pd.concat((results, get_stat_df(df,three_pt_filter,'PTS',3)))
results =pd.concat((results, get_stat_df(df,ft_filter,'PTS',1)))

results =pd.concat((results, get_stat_df(df,two_pt_filter,'FGM',1)))
results =pd.concat((results, get_stat_df(df,three_pt_filter,'3PM',1)))
results =pd.concat((results, get_stat_df(df,ft_filter,'FTM',1)))

results =pd.concat((results, get_stat_df(df,fga_filter,'FGA',1)))
results =pd.concat((results, get_stat_df(df,fta_filter,'FTA',1)))
results =pd.concat((results, get_stat_df(df,three_a_filter,'3PA',1)))

results =pd.concat((results, get_stat_df(df,trb_filter,'TRB',1)))
results =pd.concat((results, get_stat_df(df,drb_filter,'DRB',1)))
results =pd.concat((results, get_stat_df(df,orb_filter,'ORB',1)))

results =pd.concat((results, get_stat_df(df,stl_filter,'STL',1,player_num=2)))
results =pd.concat((results, get_stat_df(df,blk_filter,'BLK',1,player_num=3)))
results =pd.concat((results, get_stat_df(df,to_filter,'TOV',1,player_num=1)))
results =pd.concat((results, get_stat_df(df,foul_filter,'PF',1)))

results =pd.concat((results, get_stat_df(df,assist_filter,'AST',1,player_num='2')))

#results += apply_stat_to_lineup(df,home_poss_filter,'poss',1,a_or_v='h')#h or a
#results += apply_stat_to_lineup(df,visitor_poss_filter,'poss',1,a_or_v='a')#h or a
#results += get_time_credit_list(df)
time_credits = get_time_credit(df)
results = pd.concat((results,time_credits))
results = results.groupby(['game_id','home_poss','away_poss','player_id','player_name','h_lineup','a_lineup']).sum().reset_index()
results.loc[(results.player_name=='Stephen Curry')]

{203952: 'Andrew Wiggins', 203110: 'Draymond Green', 1626172: 'Kevon Looney', 202691: 'Klay Thompson', 201939: 'Stephen Curry', 203967: 'Dario Saric', 101108: 'Chris Paul', 1627780: 'Gary Payton II', 1630228: 'Jonathan Kuminga', 1630541: 'Moses Moody', 1631218: 'Trayce Jackson-Davis', 1641764: 'Brandin Podziemski', 202709: 'Cory Joseph', 1629622: 'Max Strus', 1630596: 'Evan Mobley', 1628386: 'Jarrett Allen', 1628378: 'Donovan Mitchell', 1629636: 'Darius Garland', 1627777: 'Georges Niang', 1627747: 'Caris LeVert', 202684: 'Tristan Thompson', 1629731: 'Dean Wade', 1630241: 'Sam Merrill', 1627745: 'Damian Jones'}


,game_id,home_poss,away_poss,player_id,player_name,h_lineup,a_lineup,eventnum,PTS,FGM,...,3PA,TRB,DRB,ORB,STL,BLK,TOV,PF,AST,play_elapsed_time
10,22300142,0,1.0,201939,Stephen Curry,"(1628378, 1628386, 1629622, 1629636, 1630596)","(201939, 202691, 203110, 203952, 1626172)",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43.0
20,22300142,0,2.0,201939,Stephen Curry,"(1628378, 1628386, 1629622, 1629636, 1630596)","(201939, 202691, 203110, 203952, 1626172)",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0
30,22300142,0,3.0,201939,Stephen Curry,"(1628378, 1628386, 1629622, 1629636, 1630596)","(201939, 202691, 203110, 203952, 1626172)",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0
40,22300142,0,4.0,201939,Stephen Curry,"(1628378, 1628386, 1629622, 1629636, 1630596)","(201939, 202691, 203110, 203952, 1626172)",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0
50,22300142,0,5.0,201939,Stephen Curry,"(1628378, 1628386, 1629622, 1629636, 1630596)","(201939, 202691, 203110, 203952, 1626172)",29.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2597,22300142,92,0.0,201939,Stephen Curry,"(1627747, 1628378, 1628386, 1629622, 1630596)","(101108, 201939, 202691, 203110, 203952)",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0
2607,22300142,93,0.0,201939,Stephen Curry,"(1627747, 1628378, 1628386, 1629622, 1630596)","(101108, 201939, 202691, 203110, 203952)",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
2618,22300142,94,0.0,201939,Stephen Curry,"(1627747, 1628378, 1628386, 1629622, 1630596)","(101108, 201939, 202691, 203110, 203952)",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0
2619,22300142,94,0.0,201939,Stephen Curry,"(1628378, 1628386, 1629622, 1629636, 1630596)","(101108, 201939, 202691, 203110, 203952)",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [190]:
threes = results.loc[(results.player_id==201939) & (results['3PA'] > 0)]
threes

,game_id,home_poss,away_poss,player_id,player_name,h_lineup,a_lineup,eventnum,PTS,FGM,...,3PA,TRB,DRB,ORB,STL,BLK,TOV,PF,AST,play_elapsed_time
90,22300142,0,8.0,201939,Stephen Curry,"(1627777, 1628378, 1628386, 1629622, 1629636)","(201939, 202691, 203110, 203952, 1626172)",192.0,3.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
110,22300142,0,10.0,201939,Stephen Curry,"(1627777, 1628378, 1628386, 1629622, 1629636)","(201939, 202691, 203110, 203952, 1626172)",118.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0
152,22300142,0,12.0,201939,Stephen Curry,"(1627747, 1627777, 1628378, 1628386, 1629622)","(201939, 202691, 203110, 203952, 203967)",300.0,3.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
227,22300142,0,16.0,201939,Stephen Curry,"(202684, 1627747, 1627777, 1628378, 1629622)","(101108, 201939, 203110, 203967, 1627780)",396.0,3.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0
530,22300142,0,36.0,201939,Stephen Curry,"(1627747, 1628378, 1628386, 1629622, 1630596)","(201939, 202691, 203110, 203952, 1626172)",1152.0,3.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0
720,22300142,0,51.0,201939,Stephen Curry,"(1628378, 1628386, 1629622, 1629636, 1630596)","(201939, 202691, 203110, 203952, 1626172)",786.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0
740,22300142,0,53.0,201939,Stephen Curry,"(1628378, 1628386, 1629622, 1629636, 1630596)","(201939, 202691, 203110, 203952, 1626172)",1608.0,3.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
760,22300142,0,55.0,201939,Stephen Curry,"(1628378, 1628386, 1629622, 1629636, 1630596)","(201939, 202691, 203110, 203952, 1626172)",826.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0
866,22300142,0,62.0,201939,Stephen Curry,"(1627747, 1627777, 1628378, 1629622, 1630596)","(201939, 202691, 203110, 203952, 1631218)",1820.0,3.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0
919,22300142,0,65.0,201939,Stephen Curry,"(1627747, 1627777, 1628378, 1629622, 1630596)","(101108, 201939, 202691, 203967, 1631218)",1896.0,3.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0


In [61]:
threes[['3PM']].rolling(10,min_periods=1).mean()

,3PM
17,1.000000
21,0.500000
27,0.666667
34,0.750000
83,0.800000
117,0.666667
120,0.714286
124,0.625000
139,0.666667
146,0.700000


In [185]:
def get_time_credit(df):
    game_id = df.game_id[0]
    box_score = pd.read_csv(f"box_scores/box_00{game_id}.csv", index_col=0)
    box = box_score[["player_id", "player_name"]].values.tolist()
    id_to_name = {id: name for id, name in box}
    print(id_to_name)
    play_times = df.groupby(['game_id','home_poss','away_poss','h_lineup','a_lineup'])["play_elapsed_time"].sum().reset_index()
    h_expl = play_times.explode('h_lineup').reset_index(drop=True)
    h_expl.rename(columns={'h_lineup':'player_id'},inplace=True)

    a_expl = play_times.explode('a_lineup').reset_index(drop=True)
    a_expl.rename(columns={'a_lineup':'player_id'},inplace=True)

    h_expl = h_expl.merge(play_times,on= ['game_id','home_poss','away_poss','a_lineup','play_elapsed_time'])
    a_expl = a_expl.merge(play_times,on= ['game_id','home_poss','away_poss','h_lineup','play_elapsed_time'])
    time_credits = pd.concat((h_expl,a_expl))
    time_credits['player_name'] = time_credits['player_id'].apply(lambda x: id_to_name[int(x)])
    time_credits['player_id'] = time_credits['player_id'].astype(int)
    return time_credits
time_credits = get_time_credit(df)
time_credits

{203952: 'Andrew Wiggins', 203110: 'Draymond Green', 1626172: 'Kevon Looney', 202691: 'Klay Thompson', 201939: 'Stephen Curry', 203967: 'Dario Saric', 101108: 'Chris Paul', 1627780: 'Gary Payton II', 1630228: 'Jonathan Kuminga', 1630541: 'Moses Moody', 1631218: 'Trayce Jackson-Davis', 1641764: 'Brandin Podziemski', 202709: 'Cory Joseph', 1629622: 'Max Strus', 1630596: 'Evan Mobley', 1628386: 'Jarrett Allen', 1628378: 'Donovan Mitchell', 1629636: 'Darius Garland', 1627777: 'Georges Niang', 1627747: 'Caris LeVert', 202684: 'Tristan Thompson', 1629731: 'Dean Wade', 1630241: 'Sam Merrill', 1627745: 'Damian Jones'}


,game_id,home_poss,away_poss,player_id,a_lineup,play_elapsed_time,h_lineup,player_name
0,22300142,0,0.0,1627745,"(202709, 1630228, 1630541, 1631218, 1641764)",17.0,"(1627745, 1629622, 1629731, 1630241, 1630596)",Damian Jones
1,22300142,0,0.0,1629622,"(202709, 1630228, 1630541, 1631218, 1641764)",17.0,"(1627745, 1629622, 1629731, 1630241, 1630596)",Max Strus
2,22300142,0,0.0,1629731,"(202709, 1630228, 1630541, 1631218, 1641764)",17.0,"(1627745, 1629622, 1629731, 1630241, 1630596)",Dean Wade
3,22300142,0,0.0,1630241,"(202709, 1630228, 1630541, 1631218, 1641764)",17.0,"(1627745, 1629622, 1629731, 1630241, 1630596)",Sam Merrill
4,22300142,0,0.0,1630596,"(202709, 1630228, 1630541, 1631218, 1641764)",17.0,"(1627745, 1629622, 1629731, 1630241, 1630596)",Evan Mobley
...,...,...,...,...,...,...,...,...
1575,22300142,100,0.0,202709,"(202709, 1630228, 1630541, 1631218, 1641764)",2.0,"(1627745, 1629622, 1629731, 1630241, 1630596)",Cory Joseph
1576,22300142,100,0.0,1630228,"(202709, 1630228, 1630541, 1631218, 1641764)",2.0,"(1627745, 1629622, 1629731, 1630241, 1630596)",Jonathan Kuminga
1577,22300142,100,0.0,1630541,"(202709, 1630228, 1630541, 1631218, 1641764)",2.0,"(1627745, 1629622, 1629731, 1630241, 1630596)",Moses Moody
1578,22300142,100,0.0,1631218,"(202709, 1630228, 1630541, 1631218, 1641764)",2.0,"(1627745, 1629622, 1629731, 1630241, 1630596)",Trayce Jackson-Davis


In [165]:
results

,game_id,home_poss,away_poss,player_id,player_name,h_lineup,a_lineup,eventnum,PTS,FGM,...,FTA,3PA,TRB,DRB,ORB,STL,BLK,TOV,PF,AST
0,22300142,0,1.0,203110,Draymond Green,"(1628378, 1628386, 1629622, 1629636, 1630596)","(201939, 202691, 203110, 203952, 1626172)",28,3.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,22300142,0,1.0,1626172,Kevon Looney,"(1628378, 1628386, 1629622, 1629636, 1630596)","(201939, 202691, 203110, 203952, 1626172)",7,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,22300142,0,1.0,1628386,Jarrett Allen,"(1628378, 1628386, 1629622, 1629636, 1630596)","(201939, 202691, 203110, 203952, 1626172)",20,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,22300142,0,1.0,1629636,Darius Garland,"(1628378, 1628386, 1629622, 1629636, 1630596)","(201939, 202691, 203110, 203952, 1626172)",9,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,22300142,0,2.0,202691,Klay Thompson,"(1628378, 1628386, 1629622, 1629636, 1630596)","(201939, 202691, 203110, 203952, 1626172)",13,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,22300142,99,0.0,1630596,Evan Mobley,"(1627745, 1629622, 1629731, 1630241, 1630596)","(202709, 1630228, 1630541, 1631218, 1641764)",1478,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
437,22300142,99,0.0,1631218,Trayce Jackson-Davis,"(1627745, 1629622, 1629731, 1630241, 1630596)","(202709, 1630228, 1630541, 1631218, 1641764)",2218,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
438,22300142,100,0.0,1630228,Jonathan Kuminga,"(1627745, 1629622, 1629731, 1630241, 1630596)","(202709, 1630228, 1630541, 1631218, 1641764)",746,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
439,22300142,100,0.0,1630541,Moses Moody,"(1627745, 1629622, 1629731, 1630241, 1630596)","(202709, 1630228, 1630541, 1631218, 1641764)",1494,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
